In [ ]:
! pip install -q t5[gcp]

     |████████████████████████████████| 163kB 3.4MB/s 
     |████████████████████████████████| 307kB 10.7MB/s 
     |████████████████████████████████| 3.6MB 11.0MB/s 
     |████████████████████████████████| 1.1MB 35.9MB/s 
     |████████████████████████████████| 1.1MB 42.8MB/s 
     |████████████████████████████████| 71kB 7.8MB/s 
     |████████████████████████████████| 2.6MB 37.9MB/s 
     |████████████████████████████████| 51kB 6.0MB/s 
     |████████████████████████████████| 5.3MB 9.3MB/s 
     |████████████████████████████████| 3.0MB 47.1MB/s 
     |████████████████████████████████| 890kB 46.7MB/s 
     |████████████████████████████████| 1.4MB 46.0MB/s 
     |████████████████████████████████| 245kB 39.3MB/s 
     |████████████████████████████████| 51kB 5.5MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
MODEL_SIZE = 'base'

DEFAULT_PATH = '/content/drive/My Drive'

TEXT_PATH = DEFAULT_PATH + '/test_text.txt'
MODEL_DIR = './output/'
GIN_CONFIG = DEFAULT_PATH + '/experiments_objectives_obj-span_3_15_u_u_operative_config.gin'

In [ ]:
import os
import tensorflow.compat.v1 as tf
import tensorflow_datasets as tfds
import t5
from t5.data import preprocessors
from t5.models import MtfModel
from t5.data.sentencepiece_vocabulary import SentencePieceVocabulary
from t5.data.utils import (
    Feature, 
    get_default_vocabulary, 
    set_global_cache_dirs,
    TaskRegistry,
    MixtureRegistry,
    TfdsTask
)
from mesh_tensorflow.transformer import utils
import functools
import gin

INFO:tensorflow:tokens_length=568 inputs_length=512 targets_length=114 noise_density=0.15 mean_noise_span_length=3.0 


In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    TPU_ADDRESS = tpu.get_master()
    #TPU_TOPOLOGY = 'v2-8'
    TPU_TOPOLOGY = '2x2'
except:
    tpu = None

print(tpu)


In [ ]:
DEFAULT_OUTPUT_FEATURES = {
    "inputs": Feature(vocabulary=get_default_vocabulary(), add_eos=True, required=False),
    "targets": Feature(vocabulary=get_default_vocabulary(), add_eos=True)      
}

gin.parse_config_file(GIN_CONFIG)

In [ ]:
def text_dataset(split, shuffle_files=False):
    del shuffle_files
    ds = tf.data.TextLineDataset(TEXT_PATH)
    ds = ds.map(lambda *ex: dict(zip(['text'], ex)))
    return ds

TaskRegistry.remove('test_pretraining')
TaskRegistry.add(
    'test_pretraining',
    dataset_fn=text_dataset,
    splits=['train'],
    text_preprocessor=functools.partial(
        preprocessors.rekey, key_map={"inputs":None, "targets":"text"}
    ),
    token_preprocessor=preprocessors.unsupervised,
    output_features=DEFAULT_OUTPUT_FEATURES,
    metric_fns=[])

t5.data.MixtureRegistry.remove("trivia_all")
t5.data.MixtureRegistry.add(
    "trivia_all",
    ['test_pretraining'],
     default_rate=1.0
)

In [ ]:
model_parallelism, train_batch_size, keep_checkpoint_max = {
    "small": (1, 256, 16),
    "base": (2, 128, 8),
    "large": (8, 64, 4),
    "3B": (8, 16, 1),
    "11B": (8, 16, 1)}[MODEL_SIZE]

tf.io.gfile.makedirs(MODEL_DIR)
# The models from our paper are based on the Mesh Tensorflow Transformer.

if tpu:
    model = MtfModel(
        model_dir=MODEL_DIR,
        tpu=TPU_ADDRESS,
        tpu_topology=TPU_TOPOLOGY,
        model_parallelism=model_parallelism,
        batch_size=train_batch_size,
        learning_rate_schedule=1e-5,
        keep_checkpoint_max=keep_checkpoint_max,
        iterations_per_loop=100
    )
else:
    model = MtfModel(
        model_dir=MODEL_DIR,
        tpu=tpu,
        model_parallelism=model_parallelism,
        batch_size=train_batch_size,
        learning_rate_schedule=1e-5,
        keep_checkpoint_max=keep_checkpoint_max,
        iterations_per_loop=100
    )

model.train(mixture_or_task_name='trivia_all', steps=1000)

INFO:tensorflow:Using config: {'_model_dir': './output/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.98.250.170:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({'worker': ['10.98.250.170:8470']}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.98.250.170:8470', '_evaluation_master': 'grpc://10.98.250.170:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': 

/usr/local/lib/python3.6/dist-packages/t5/data/utils.py:273: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  return dataset.map(my_fn, num_parallel_calls=tf.data.experimental.AUTOTUNE)


INFO:tensorflow:tokens_length=568 inputs_length=512 targets_length=114 noise_density=0.15 mean_noise_span_length=3.0 
INFO:tensorflow:num_cores_per_replica: 1
INFO:tensorflow:computation_shape: [1, 1, 1, 1]
INFO:tensorflow:num_replicas: 8
INFO:tensorflow:device_assignment.topology.device_coordinates: [[[0 0 0 0]
  [0 0 0 1]
  [1 0 0 0]
  [1 0 0 1]
  [0 1 0 0]
  [0 1 0 1]
  [1 1 0 0]
  [1 1 0 1]]]
INFO:tensorflow:device_assignment.core_assignment: [[[0 0 0 0]]

 [[0 0 0 1]]

 [[1 0 0 0]]

 [[1 0 0 1]]

 [[0 1 0 0]]

 [[0 1 0 1]]

 [[1 1 0 0]]

 [[1 1 0 1]]]
INFO:tensorflow:auto_logical_to_physical_tpu logical_shape=[4, 2] physical_shape=[2, 2, 2]
INFO:tensorflow:auto_logical_to_physical_tpu logical_shape=[2] physical_shape=[1, 1, 2]
INFO:tensorflow:auto_logical_to_physical_tpu logical_to_physical = [(0, 0, 0), (0, 0, 1)]
INFO:tensorflow:auto_logical_to_physical_tpu logical_to_physical = [(0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 1, 0), (1, 1, 1), (1, 0, 0), (1, 0, 1)]
INFO:tensorf

ValueError: ignored